In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import networkx
import matplotlib.pyplot as plt 

from ws_crl_lite.datasets.dataset import build_manual_train_test

RuntimeError: KeyboardInterrupt: <EMPTY MESSAGE>

In [ ]:
train, test = build_manual_train_test(101, 50, "atomic_markov1", 42)